In [7]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# 規則
- 使用しないデータ：0
- トレーニングデータ：1
- テストデータ：2

# パターン1
## トレーニングデータ：trainデータ，テストデータ：validationデータ

In [2]:
train = np.load("../Data/センサ別npyファイル/train/train_Bag/train_Bag_Label.npy")
validation = np.load("../Data/センサ別npyファイル/validation/validation_Bag/validation_Bag_Label.npy")

train.shape, validation.shape

((195491, 500, 1), (28685, 500))

In [3]:
train = train[:, 0]
validation = validation[:, 0]

# パターン2
## トレーニングデータ：ユーザ1,2,3それぞれ前5割
## テストデータ：ユーザ1,2,3それぞれ後3割

In [4]:
train.shape[0] // 2 + 182

97927

In [5]:
train.shape[0] - train.shape[0] // 10 * 7

58648

In [6]:
136843 - 97927

38916

In [7]:
train_pattern2 = [1] * 97927 + [0] * 38916 + [2] * 58648
train_pattern2 = np.array(train_pattern2).reshape([-1, 1])
train_pattern2.shape

(195491, 1)

#### 120845を消す(NaNなので)

In [8]:
train_pattern2 = np.delete(train_pattern2, 120845, 0)
train_pattern2.shape

(195490, 1)

In [9]:
np.save("train_pattern2", train_pattern2)

In [1]:
14813 // 2

7406

In [10]:
14813 // 10 * 10

14810

In [4]:
7406 + 2962

10368

In [14]:
13872 // 10 * 3

4161

In [16]:
validation_pattern2 = [1] * 7406 + [0] * 2962 + [2] * 4445 + [1] * 6936 + [0] * 2774 + [2] * 4162
validation_pattern2 = np.array(validation_pattern2).reshape([-1, 1])
validation_pattern2.shape

(28685, 1)

In [17]:
np.save("validation_pattern2", validation_pattern2)

# パターン3
## トレーニングデータ：ユーザ2データ
## テストデータ：ユーザ1データ（trainデータ），ユーザ3データ

In [10]:
train_pattern3 = [2] * train.shape[0]
train_pattern3 = np.array(train_pattern3).reshape([-1, 1])
train_pattern3.shape

(195491, 1)

#### 120845消す

In [12]:
train_pattern3 = np.delete(train_pattern3, 120845, 0)
train_pattern3.shape

(195490, 1)

In [13]:
np.save("train_pattern3", train_pattern3)

In [47]:
validation_pattern3 = [1] * 14813 + [2] * 13872
validation_pattern3 = np.array(validation_pattern3).reshape([-1, 1])
validation_pattern3.shape

(28685, 1)

In [48]:
np.save("validation_pattern3", validation_pattern3)

# パターン4（パターン2の逆）
## トレーニングデータ：ユーザ1,2,3それぞれ後3割
## テストデータ：ユーザ1,2,3それぞれ前5割

In [14]:
train_pattern4 = [2] * 97927 + [0] * 38916 + [1] * 58648
train_pattern4 = np.array(train_pattern4).reshape([-1, 1])
train_pattern4.shape

(195491, 1)

In [15]:
train_pattern4 = np.delete(train_pattern4, 120845, 0)
train_pattern4.shape

(195490, 1)

In [16]:
np.save("train_pattern4", train_pattern4)

In [18]:
validation_pattern4 = [2] * 7406 + [0] * 2962 + [1] * 4445 + [2] * 6936 + [0] * 2774 + [1] * 4162
validation_pattern4 = np.array(validation_pattern4).reshape([-1, 1])
validation_pattern4.shape

(28685, 1)

In [19]:
np.save("validation_pattern4", validation_pattern4)

# パターン5（パターン3のユーザ2と3を入れ替えた感じ）
## トレーニングデータ：ユーザ3データ
## テストデータ：ユーザ1データ（Trainデータ），ユーザ2データ

In [17]:
train_pattern5 = [2] * train.shape[0]
train_pattern5 = np.array(train_pattern5).reshape([-1, 1])
train_pattern5.shape

(195491, 1)

In [18]:
train_pattern5 = np.delete(train_pattern5, 120845, 0)
train_pattern5.shape

(195490, 1)

In [19]:
np.save("train_pattern5", train_pattern5)

In [55]:
validation_pattern5 = [2] * 14813 + [1] * 13872
validation_pattern5 = np.array(validation_pattern5).reshape([-1, 1])
validation_pattern5.shape

(28685, 1)

In [56]:
np.save("validation_pattern5", validation_pattern5)

# 新パターン2
## ユーザ1全部とユーザ2,3各ラベル前5割を学習して良いデータ
## ユーザ2,3の各ラベル後3割を検証データ

In [1]:
import numpy as np

from sklearn.model_selection import train_test_split

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
val_Label = np.load("../Data/センサ別npyファイル/validation/validation_Bag/validation_Bag_Label.npy")[:, 0].reshape([-1])
val_Label.shape

(28685,)

In [4]:
user2_Label = val_Label[:14813]
user2_Label.shape

(14813,)

In [5]:
pattern2_user2 = np.zeros(14813)
pattern2_user2.shape

(14813,)

In [29]:
learn_size = 0
test_size = 0
for i in range(8):
    tmp = np.where(user2_Label == (i+1))[0]
    learn = tmp[:tmp.shape[0]//2]
    test = tmp[tmp.shape[0]//10*7:]
    print(learn.shape, test.shape)
    learn_size += len(learn)
    test_size += len(test)
    
print(learn_size, test_size)

(1205,) (724,)
(1470,) (883,)
(90,) (54,)
(817,) (493,)
(1541,) (926,)
(419,) (257,)
(805,) (483,)
(1058,) (640,)
7405 4460


In [31]:
np.where(pattern2_user2 == 0)[0].shape

(2948,)

In [33]:
user3_Label = val_Label[14813:]
user3_Label.shape

(13872,)

In [34]:
pattern2_user3 = np.zeros(13872)
pattern2_user3.shape

(13872,)

In [36]:
learn_size = 0
test_size = 0
for i in range(8):
    tmp = np.where(user3_Label == (i+1))[0]
    learn = tmp[:tmp.shape[0]//2]
    test = tmp[tmp.shape[0]//10*7:]
    print(learn.shape, test.shape)
    learn_size += len(learn)
    test_size += len(test)
    pattern2_user3[learn] = 1
    pattern2_user3[test] = 2
    
print(learn_size, test_size)

(1762,) (1061,)
(1124,) (681,)
(185,) (112,)
(383,) (234,)
(503,) (307,)
(496,) (299,)
(1371,) (825,)
(1109,) (672,)
6933 4191


In [40]:
np.where(pattern2_user3 == 0)[0].shape

(2748,)

In [41]:
pattern2 = np.concatenate([pattern2_user2, pattern2_user3], axis=0)
pattern2.shape

(28685,)

In [42]:
np.save("validation_pattern2", pattern2)